# Amplitude embedding (simulator)

Main steps:

* initialize an image of the sky (frequency domain)
* apply 2D FT --> visibilities (Fourier domain)
* encode visibilities data into qubits using amplitude embedding (non-conventional domain)
* measure qubits on simulator --> expected outcomes (back to conventional domain)
* apply 2D IFT --> fidelity computation

In [67]:
import numpy as np
import math
import sys

import pennylane as qml
from pennylane import numpy as pnp

## 1) Generating an artificial image of the sky (frequency domain)

In [68]:
#image of the sky filled with double precision complex floats ('complex64')
#pixels are set to low complex random values (image background/noise) 
#few pixels are set to larger complex random values in a specified ellipse area (image source/subject)

n = 4
image = np.zeros((n, n), dtype='complex64')
image.real = np.random.rand(n , n) / 100
image.imag = np.random.rand(n , n) / 100

h, w = image.shape
mask = circular_mask(h, w, radius=h/2)
sky_image = image.copy()
sky_image[~mask] = complex(np.random.rand() * 100, np.random.rand() * 100)
print(sky_image)

[[3.9988861e+00+6.3125386e+00j 3.9988861e+00+6.3125386e+00j
  5.5041946e-03+3.4709403e-03j 3.9988861e+00+6.3125386e+00j]
 [3.9988861e+00+6.3125386e+00j 1.4900982e-03+1.7262183e-03j
  9.8942043e-03+6.3721329e-04j 9.0795523e-03+3.5714540e-03j]
 [8.6654015e-03+4.8104352e-03j 7.8281984e-03+5.5479975e-03j
  2.0459113e-03+9.2881862e-03j 5.8600237e-03+6.3649914e-03j]
 [3.9988861e+00+6.3125386e+00j 4.9103093e-03+3.3015772e-03j
  6.7689535e-03+8.2479315e-03j 4.9895532e-03+4.2807129e-03j]]


## 2) Applying a 2D FT (Fourier domain)

In [69]:
visibilities = np.fft.fft2(sky_image).astype('complex64')
print(visibilities)

[[ 20.061466 +31.613941j   11.977469 +18.926483j    3.997607  +6.3142004j
   11.984752 +18.915081j ]
 [ 11.967868 +18.91128j     4.0027003 +6.3195047j  -3.995731  -6.313709j
    3.9860458 +6.3138366j]
 [  3.9916573 +6.320256j   -3.9791002 -6.321239j  -11.990325 -18.927965j
   -3.983115  -6.301965j ]
 [ 11.9876585+18.91887j     3.9724586 +6.3115225j  -3.9850788 -6.308798j
    3.9858449 +6.309318j ]]


#### Sanity check

In [70]:
img = np.fft.ifft2(visibilities).astype('complex64')
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()
 
print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[3.9988861e+00+6.3125386e+00j 3.9988861e+00+6.3125386e+00j
  5.5041462e-03+3.4709275e-03j 3.9988861e+00+6.3125386e+00j]
 [3.9988861e+00+6.3125386e+00j 1.4900416e-03+1.7262697e-03j
  9.8942220e-03+6.3727796e-04j 9.0796202e-03+3.5713613e-03j]
 [8.6653382e-03+4.8104227e-03j 7.8280866e-03+5.5480748e-03j
  2.0457953e-03+9.2881620e-03j 5.8600605e-03+6.3650459e-03j]
 [3.9988861e+00+6.3125386e+00j 4.9100965e-03+3.3015311e-03j
  6.7688525e-03+8.2479864e-03j 4.9894899e-03+4.2806864e-03j]]

Reals MSE: 6.3305243e-15
Imaginaries MSE: 1.9448658e-15


## 3) Data encoding: amplitude embedding (non-conventional domain)

In [71]:
# Amplitude embedding encodes a normalized 2^n-dimensional feature vector into the state of n qbits (uses log2(n) qbits for n classical data)
n = visibilities.shape[0]*visibilities.shape[1]
data = visibilities.flatten()

#normalization to prepare a qstate with measurement probabilites summing up to 1 (SUM (amplitudes²) = 1)
norm = qml.math.sum(qml.math.abs(data) ** 2)
normalized_data = data / qml.math.sqrt(norm)

wires = range(int(math.log2(n))) # set the number of qbits (no padding needed if outputs an integer=integer.0)
amp_dev = qml.device('default.qubit', wires)

@qml.qnode(amp_dev)
def amp_encoding(data):
    qml.AmplitudeEmbedding(data, wires) # normalize = True
    return qml.state() #qml.expval(qml.PauliZ(wires=wires))

results = amp_encoding(normalized_data).astype('complex64')
print(results)

print()

results = results*qml.math.sqrt(norm) # denormalization of the measurements outcomes
results = np.array(results).reshape(sky_image.shape[0] , sky_image.shape[1])
print(results)

[ 0.30015621+0.47300234j  0.17920484+0.28317478j  0.05981151+0.09447198j
  0.17931379+0.28300419j  0.17906117+0.2829473j   0.05988771+0.09455134j
 -0.05978344-0.09446462j  0.05963853+0.09446654j  0.05972249+0.09456258j
 -0.05953461-0.09457729j -0.17939718-0.28319696j -0.05959468-0.09428892j
  0.17935728+0.28306088j  0.05943524+0.09443191j -0.05962406-0.09439114j
  0.05963552+0.09439893j]

[[ 20.061466 +31.613941j   11.977469 +18.926483j    3.997607  +6.3142004j
   11.984752 +18.915081j ]
 [ 11.967868 +18.91128j     4.0027003 +6.3195047j  -3.995731  -6.313709j
    3.9860458 +6.3138366j]
 [  3.9916573 +6.320256j   -3.9791002 -6.321239j  -11.990325 -18.927965j
   -3.983115  -6.301965j ]
 [ 11.9876585+18.91887j     3.9724586 +6.3115225j  -3.9850786 -6.308798j
    3.9858449 +6.309318j ]]


## 4) Applying 2D IFT (fidelity test)

In [72]:
img = np.fft.ifft2(results).astype('complex64')
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()

print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[3.9988861e+00+6.3125386e+00j 3.9988861e+00+6.3125386e+00j
  5.5041611e-03+3.4709275e-03j 3.9988861e+00+6.3125386e+00j]
 [3.9988861e+00+6.3125386e+00j 1.4900416e-03+1.7262846e-03j
  9.8942220e-03+6.3726306e-04j 9.0796202e-03+3.5713762e-03j]
 [8.6653233e-03+4.8104227e-03j 7.8281015e-03+5.5480748e-03j
  2.0457804e-03+9.2881620e-03j 5.8600754e-03+6.3650459e-03j]
 [3.9988861e+00+6.3125386e+00j 4.9100965e-03+3.3015162e-03j
  6.7688525e-03+8.2480013e-03j 4.9894899e-03+4.2806715e-03j]]

Reals MSE: 6.5039967e-15
Imaginaries MSE: 2.0685733e-15


## Utils

### Quantum

### Classical

In [73]:
#creates a circular mask over a 2D array
def circular_mask(h, w, center=None, radius=None):
    if center is None: #image center
        center = (int(w/2), int(h/2))
    if radius is None: #smallest distance between center and image bounderies
        radius = min(center[0], center[1], w-center[0], h-center[1])
        
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
    mask = dist_from_center <= radius
    
    return mask